In [1]:
import math 

def discretize(waiting_times):
    for waiting_time in waiting_times:
        if waiting_time > 99.9:
            waiting_time = 99
    return 10000000* abs(round(waiting_times[0])) + 10000*abs(round(waiting_times[1])) + 10*abs(round(waiting_times[2]))

In [2]:
import gym
from gym import spaces
import numpy as np
from math import ceil

class TrafficGenerator(gym.Env):
    
    #doesnt really workj
    def getObs(self):
        # Loop over the packet types
        for i in range(len(self.mean_delay)):

            # Get the current queue for the packet type
            current_queue = self.queues[i]

            # Calculate the length and average waiting time of the current queue
            current_length = len(current_queue)
            current_waiting_time = np.average(current_queue) if current_length > 0 else 0.0
            #print(np.average(current_queue), current_length, current_waiting_time )
            
            
           
            observation.append([current_length, current_waiting_time])

       
        observation = np.array(observation)
        return observation
    
    def __init__(self):
        
        self.action_space = spaces.Discrete(3)
        
        # Define the observation space (number of packets in each queue and their waiting time)
        # Queue maximum length set to 100
        self.observation_space = spaces.Discrete(1000000000)
        
        # pancket info (DataType, arrival_rate, mean_delay )
        #self.packetInfo = [[0, 0.3, 6],[1, 0.25, 4],[2, 0.4, float('inf')]]
        
        self.dataType = [0, 1, 2]
        self.arrival_rate = [0.3, 0.25, 0.4]
        self.mean_delay_req = [6, 4, float('inf')]
        self.packetInfo = [[elem1, elem2, elem3] for elem1, elem2, elem3 in zip(self.dataType, self.arrival_rate, self.mean_delay_req)]
        self.curr_mean_delay_best_effort = 0
        self.packet = 1
        self.timeslot = 1
        self.totaltime = self.timeslot
        
        # Initialize the queues
        self.queues = [[], [], []]
      
    
    def step(self, action):  
        self.totaltime += self.timeslot
        #print(self.totaltime)
        #print("~~~~~~~~~~~~~~~")
        #print("Action Chosen: ", action)
       

        #OBSERVING ENVRONEMNT
        observation = []
        for i in range(len(self.mean_delay_req)):

            # Get the current queue for the packet type
            current_queue = self.queues[i]

            # Calculate the length and average waiting time of the current queue
            current_length = len(current_queue)
            current_waiting_time = np.average(current_queue) if current_length > 0 else 0.0
            #print(np.average(current_queue), current_length, current_waiting_time )
            current_waiting_time = np.round(current_waiting_time, 1)   
            observation.append([current_length, current_waiting_time])
            if i == 2:
                #reward option
                if self.curr_mean_delay_best_effort == 0:
                    curr_mean_delay_best_effort = current_waiting_time
                if self.curr_mean_delay_best_effort < current_waiting_time:
                    self.curr_mean_delay_best_effort = current_waiting_time
                    reward = 1
                elif self.curr_mean_delay_best_effort > current_waiting_time:
                    self.curr_mean_delay_best_effort = current_waiting_time
                    reward = -1
                else:
                    reward = 1
        
                #print(self.curr_mean_delay_best_effort)
                #print(current_waiting_time)


        observation = np.array(observation)  
        
        #Discoruge it from choosing if its not close to the requirement
        #print(f"observation{observation}")
        if observation[0][1] < -4 and action != 0:
            reward = -10
        elif observation[1][1] < -6 and action != 1:
            reward = -10
        
        if (len(self.queues[action]) == 0):
            reward = -25
            #print("LOL STUPID LENGTH IS:", len(self.queues[action])  )
        
        if not any(self.queues):
                reward = 25
                #print(\"Empty list!\")\n",
        
        #UPDATING ENVIORNMENT
        
        
        # services the queues
        #for i in range(len(self.queues)):
            #print("queue ", i, ": has a length of", len(self.queues[i]))
        #print("ACTION queue ", action, ": has a length of", len(self.queues[action]))
        if action < 3 and len(self.queues[action]) > 0:
            self.queues[action].pop(0)

        
        
        #packet generator
        for i in range(len(self.packetInfo)):
            for sublist in self.packetInfo:
                if sublist[0] == i and np.random.uniform() < sublist[1]:
                    self.queues[i].append(self.packet)
                    print("Appending to queue", i)
        
        #adding time
        for i in range(len(self.queues)):
            for j in range(len(self.queues[i])):
                self.queues[i][j] -= self.timeslot
        
        #Updating the new observaiton 
        observationUpdate = []
        for i in range(len(self.mean_delay_req)):

            # Get the current queue for the packet type
            current_queue = self.queues[i]

            # Calculate the length and average waiting time of the current queue
            current_length = len(current_queue)
            current_max_time = np.min(current_queue) if current_length > 0 else 0.0
            #print(np.average(current_queue), current_length, current_waiting_time )
            #current_waiting_time = np.round(current_waiting_time, 1)
            
            observationUpdate.append([current_length, current_max_time])      
        
        observationUpdate = np.array(observationUpdate) 

              
       
        info = {"Mean Delay Req 0": self.mean_delay_req[0],
        "Mean Delay Req 1": self.mean_delay_req[1],
        "Mean Delay Req 2": self.mean_delay_req[2],
        "arrival_rate_0": self.arrival_rate[0],
        "arrival_rate_1": self.arrival_rate[1],
        "arrival_rate_2": self.arrival_rate[2]}
        #done = len(self.queues[0]) + len(self.queues[1]) + len(self.queues[2]) == 0
        done = (self.totaltime >= 100)
        
        #print(f"WAITING TIMES {np.array(waiting_times)}")
        #waiting_times = discretize(waiting_times)
        
        
        
        return observationUpdate, reward, done, info
        
    def reset(self):
        # Reset the queues
        self.queues = [[], [], []]
        self.totaltime = 1
        return np.array([[len(self.queues[i]), 0.0] for i in range(len(self.mean_delay_req))])
        
    def render(self):
        pass

In [3]:
#FIFO
env = TrafficGenerator()
obs = env.reset()
done = False
x = 0
totalReward = 0
while x < 100:
    print("~~~~~~~~~")
    #print(type(obs))
    print(obs)
    print("Actions to choose form", obs[:3, 1]) 
    action = np.argmin(obs[:3, 1])
    
    if (np.any(obs[:3,0]) and not np.any(obs[:3,1] )):
        action = np.argmax(obs[:3, 0])
    
    print("chosen", action)
    obs,reward, done, info = env.step(action)
    
    totalReward += reward
    print("reward", reward)
    
    x = x +1
print(totalReward)

~~~~~~~~~
[[0. 0.]
 [0. 0.]
 [0. 0.]]
Actions to choose form [0. 0. 0.]
chosen 0
Appending to queue 2
reward 25
~~~~~~~~~
[[0. 0.]
 [0. 0.]
 [1. 0.]]
Actions to choose form [0. 0. 0.]
chosen 2
reward 1
~~~~~~~~~
[[0. 0.]
 [0. 0.]
 [0. 0.]]
Actions to choose form [0. 0. 0.]
chosen 0
reward 25
~~~~~~~~~
[[0. 0.]
 [0. 0.]
 [0. 0.]]
Actions to choose form [0. 0. 0.]
chosen 0
Appending to queue 0
Appending to queue 1
Appending to queue 2
reward 25
~~~~~~~~~
[[1 0]
 [1 0]
 [1 0]]
Actions to choose form [0 0 0]
chosen 0
Appending to queue 1
Appending to queue 2
reward 1
~~~~~~~~~
[[ 0.  0.]
 [ 2. -1.]
 [ 2. -1.]]
Actions to choose form [ 0. -1. -1.]
chosen 1
Appending to queue 0
Appending to queue 1
reward -1
~~~~~~~~~
[[ 1  0]
 [ 2 -1]
 [ 2 -2]]
Actions to choose form [ 0 -1 -2]
chosen 2
reward -1
~~~~~~~~~
[[ 1 -1]
 [ 2 -2]
 [ 1 -2]]
Actions to choose form [-1 -2 -2]
chosen 1
Appending to queue 2
reward -1
~~~~~~~~~
[[ 1 -2]
 [ 1 -2]
 [ 2 -3]]
Actions to choose form [-2 -2 -3]
chosen 2
rewa